# Results Aggregation and Paper Generation

This notebook:
1. Aggregates metrics from all experiments
2. Generates comparison tables
3. Creates publication-ready figures
4. Fills LaTeX template with results
5. Compiles ICML 2025 short paper PDF

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

sys.path.insert(0, str(Path('..').resolve()))

from src.evaluation.aggregate import aggregate_results, generate_figures

ARTIFACTS = Path('../artifacts')
PAPER_DIR = Path('../paper')
OUTPUT_DIR = PAPER_DIR / 'output'

sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 150

## Load All Metrics

In [ ]:
# Load metrics from all experiments
metrics_files = list((ARTIFACTS / 'metrics').glob('*.json'))
all_metrics = []

for f in metrics_files:
    with open(f) as fh:
        metrics = json.load(fh)
        metrics['experiment'] = f.stem
        all_metrics.append(metrics)

results_df = pd.DataFrame(all_metrics)
print(f"Loaded {len(results_df)} experiment results")
print(results_df.head())

## Generate Summary Tables

In [ ]:
# Create comparison table
summary = results_df.groupby(['modality', 'model', 'dataset']).agg({
    'test_accuracy': 'mean',
    'test_f1_macro': 'mean'
}).round(4)

print(summary)

# Save as LaTeX
latex_table = summary.to_latex()
with open(PAPER_DIR / 'tables/results_summary.tex', 'w') as f:
    f.write(latex_table)

## Generate Figures

In [ ]:
# Training curves
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
# Plot training curves for each model
# Save to paper/figures/

plt.tight_layout()
plt.savefig(PAPER_DIR / 'figures/training_curves.pdf', bbox_inches='tight')
plt.show()

## Generate Confusion Matrices

In [ ]:
# Load and plot confusion matrices
# Save to paper/figures/

## Fill LaTeX Template

In [ ]:
# Read template and fill with results
template_path = Path('../icml2025-template.tex')
with open(template_path) as f:
    template = f.read()

# Replace placeholders with actual values
# Save to paper/icml2025_bird_classification.tex

## Compile PDF

In [ ]:
import subprocess

# Compile LaTeX to PDF
result = subprocess.run(
    ['pdflatex', '-output-directory', str(OUTPUT_DIR), str(PAPER_DIR / 'icml2025_bird_classification.tex')],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("PDF compiled successfully!")
    print(f"Output: {OUTPUT_DIR / 'icml2025_bird_classification.pdf'}")
else:
    print("Compilation failed:")
    print(result.stderr)